In [1]:
import os
import keras
import warnings
import tensorflow as tf
import numpy as np
import pandas as pd
import multiprocessing
from scipy import signal
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Conv1D, Conv1DTranspose, MaxPooling1D, AveragePooling1D, Dense, BatchNormalization, Activation, Add, Flatten, Dropout, Concatenate, LeakyReLU
print(f'Is GPU Avaliable: {tf.config.list_physical_devices("GPU")}')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.filterwarnings(action='ignore')

DATA_PATH = '/root/Workspace/DataWarehouse/stMary_RRpo'
DATA_SAVE_PATH = '/root/Workspace/Project-RRpo-2ndStudy/dataset' 

2023-09-25 18:28:51.945775: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-25 18:28:51.998905: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-25 18:28:52.789848: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Is GPU Avaliable: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
EPOCHS = 1000
BATCH_SIZE = 256
LR = 0.001
callbacks = [
    EarlyStopping(monitor='val_loss', patience=33),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)
]

In [3]:
class DResIncBlock(Model):
    def __init__(self, starting_filter=32, dropout=0, *args, **kwargs):
        super(DResIncBlock, self).__init__(*args, **kwargs)
        self.conv11 = Conv1D(filters=starting_filter, kernel_size=1, strides=1, padding='same')
        self.bn11 = BatchNormalization()

        self.conv21 = Conv1D(filters=starting_filter, kernel_size=1, strides=1, padding='same')
        self.bn21 = BatchNormalization()
        self.conv22 = Conv1D(filters=starting_filter, kernel_size=3, strides=1, dilation_rate=2, padding='same')
        self.bn22 = BatchNormalization()

        self.conv31 = Conv1D(filters=starting_filter, kernel_size=1, strides=1, padding='same')
        self.bn31 = BatchNormalization()
        self.conv32 = Conv1D(filters=starting_filter, kernel_size=3, strides=1, dilation_rate=4, padding='same')
        self.bn32 = BatchNormalization()

        self.conv41 = Conv1D(filters=starting_filter, kernel_size=1, strides=1, padding='same')
        self.bn41 = BatchNormalization()
        self.conv42 = Conv1D(filters=starting_filter, kernel_size=3, strides=1, dilation_rate=8, padding='same')
        self.bn42 = BatchNormalization()

    @tf.function
    def call(self, inputs, training=None, mask=None):
        # inputs ?, timestep, features
        x1 = self.conv11(inputs) 
        x1 = self.bn11(x1)
        
        x2 = self.conv21(inputs)
        x2 = self.bn21(x2)
        x2 = Activation('relu')(x2)
        x2 = self.conv22(x2)
        x2 = self.bn22(x2)
        
        x3 = self.conv31(inputs)
        x3 = self.bn31(x3)
        x3 = Activation('relu')(x3)
        x3 = self.conv32(x3)
        x3 = self.bn32(x3)
        
        x4 = self.conv41(inputs)
        x4 = self.bn41(x4)
        x4 = Activation('relu')(x4)
        x4 = self.conv42(x4)
        x4 = self.bn42(x4)
        
        x = Concatenate()([x1, x2, x3, x4]) # ?, input_timestep, features * 4
        outputs = Add()([x, inputs])
        return outputs


class ContractingPath(Model):
    def __init__(self, filters, *args, **kwargs):
        super(ContractingPath, self).__init__(*args, **kwargs)
        self.blocks = [DResIncBlock(starting_filter=filters*(2**i)) for i in range(3)]
        self.dwnconv = [Conv1D(filters=1, kernel_size=2, strides=2, padding='same') for _ in range(3)]
        self.bn = [BatchNormalization() for _ in range(3)]

    @tf.function
    def call(self, inputs, training=None, mask=None):
        f_map1 = self.blocks[0](inputs) # ?, 1800, 128(64)
        x1 = self.dwnconv[0](f_map1) 
        x1 = self.bn[0](x1)
        x1 = LeakyReLU(alpha=0.2)(x1) # ?, 900, 128(64)

        f_map2 = self.blocks[1](x1) # ?, 900, 256(128)
        x2 = self.dwnconv[1](f_map2)
        x2 = self.bn[1](x2)
        x2 = LeakyReLU(alpha=0.2)(x2) # ?, 450, 256(128)

        f_map3 = self.blocks[2](x2) # ?, 450, 512(256)
        x3 = self.dwnconv[2](f_map3)
        x3 = self.bn[2](x3)
        x3 = LeakyReLU(alpha=0.2)(x3) # ?, 225, 512(256)

        return x3, f_map1, f_map2, f_map3
    

class ExpandingPath(Model):
    def __init__(self, filters=512, *args, **kwargs):
        super(ExpandingPath, self).__init__(*args, **kwargs)
        self.upconv = [Conv1DTranspose(filters=int(filters/(2**i)), kernel_size=2, strides=2, padding='same') for i in range(3)]
        self.blocks = [DResIncBlock(starting_filter=int((filters/(2**i))/4)) for i in range(3)]
        self.bn = [BatchNormalization() for _ in range(3)]
    
    @tf.function
    def call(self, inputs, feature_map, training=None, mask=None):
        # (?, 225, 512)
        x = self.upconv[2](inputs) # (?, 450, 512)
        x = self.bn[2](x)
        x = LeakyReLU(alpha=0.2)(x)
        x = self.blocks[2](x) # (?, 450, 512)
        x = Concatenate()([x, feature_map[2]]) # (?, 450, 1024)

        x = self.upconv[1](x) # (?, 900, 256)
        x = self.bn[1](x)
        x = LeakyReLU(alpha=0.2)(x)
        x = self.blocks[1](x) # (?, 900, 256)
        x = Concatenate()([x, feature_map[1]]) # (?, 900, 512)

        x = self.upconv[0](x) # (?, 1800, 128)
        x = self.bn[0](x)
        x = LeakyReLU(alpha=0.2)(x)
        x = self.blocks[0](x) # (?, 1800, 128)
        x = Concatenate()([x, feature_map[0]]) # (?, 1800, 256)

        return x
    

class RespNet(Model):
    def __init__(self, *args, **kwargs):
        super(RespNet, self).__init__(*args, **kwargs)
        self.cpath = ContractingPath(filters=16)
        self.bottleneck = DResIncBlock(starting_filter=64)
        self.xpath = ExpandingPath(filters=256)
        self.conv1 = Conv1D(filters=1, kernel_size=2, strides=1, padding='valid')
        self.bn1 = BatchNormalization()

        self.d100 = Dense(100, activation='relu')
        self.d50 = Dense(50, activation='relu')
        self.d10 = Dense(10, activation='relu')
        self.d1 = Dense(1)

    @tf.function
    def call(self, inputs, training=None, mask=None):
        x = inputs # ?, 1800, 1
        x, f1, f2, f3 = self.cpath(x) # (?, 225, 512), (?, 1800, 128), (?, 900, 256), (?, 450, 512)
        x = self.bottleneck(x) # (?, 225, 512)
        x = self.xpath(x, (f1, f2, f3)) # (?, 1800, 256)

        print(x.shape)
        # x = self.conv1(x) # (?, 1800, 1) # 여기가 문제인 듯? 위에 x는 샘플?의 2차원 행렬일텐데 그걸 1차원 conv로 풀려고 하면 256번 하겠지, stride=2에 valid니까 timestep은 1799일테고,
        # x = self.bn1(x)
        # x = LeakyReLU(alpha=0.2)(x)

        x = Flatten()(x)
        x = self.d100(x)
        x = self.d50(x)
        x = self.d10(x)
        print(x.shape)
        return self.d1(x)

In [4]:
def gen_tfdataset(dataset, batchsize):
    X = []; y = []
    for pleth, resp in dataset:
        X.append(pleth.astype(np.float32))
        y.append(resp)

    X = np.array(X); y = np.array(y).reshape(-1,1)
    scaler = MinMaxScaler()
    scaled_X = np.asarray([scaler.fit_transform(pleth.reshape(-1,1)) for pleth in X])
    print(f'Overall: {scaled_X.shape}, {y.shape}')
    return tf.data.Dataset.from_tensor_slices((scaled_X, y)).batch(batchsize)


def cross_validation(model, model_name, dataset, dataset_name, n_splits=5, batch_size=256, lr=0.001):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    # kf = KFold(n_splits=n_splits)
    train_losses = []; val_losses = []
    subject_id = np.array([subject[0] for subject in dataset])
    count = 1
    
    for train_idx, val_idx in kf.split(subject_id):
        obj_model = model()
        # obj_model = model(num_of_blocks=4, kernel_size=2, dilation_rate=3, dwn_kernel_size=3, filters=8, strides_of_avg=2, units=86)
        
        train_dataset = []; val_dataset = []
        for id, samples in dataset:
                if id in subject_id[train_idx]: train_dataset.extend(samples)
                else: val_dataset.extend(samples)
        
        train_dataset = np.array(train_dataset); val_dataset = np.array(val_dataset)

        train_tf = gen_tfdataset(train_dataset, batch_size)
        val_tf = gen_tfdataset(val_dataset, batch_size)

        obj_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
            loss=keras.losses.MeanAbsoluteError(),
            metrics=keras.metrics.MeanAbsoluteError()
        )  

        history = obj_model.fit(
            train_tf,
            epochs=EPOCHS,
            callbacks=callbacks,
            validation_data=val_tf
        )

        min_val_loss_idx = np.argmin(history.history['val_loss'])
        train_losses.append(history.history['loss'][min_val_loss_idx])
        val_losses.append(history.history['val_loss'][min_val_loss_idx])
        count = count + 1
        break
    
    print(f'TRAIN: {np.mean(train_losses)} ± {np.std(train_losses)}')
    print(f'VAL: {np.mean(val_losses)} ± {np.std(val_losses)}')
    return obj_model, train_losses, val_losses

In [5]:
stmary = np.load(f'{DATA_SAVE_PATH}/230920/stmary-trainval_dataset.npy', allow_pickle=True)
print(stmary.shape)

(85, 2)


In [6]:
model = RespNet()

2023-09-25 18:28:56.443673: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13672 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:73:00.0, compute capability: 7.5


In [7]:
p = []; r =[]
for id, samples in stmary:
    for pleth, resp in samples:
        p.append(pleth.reshape(1,-1,1))
        r.append(resp)

p = np.concatenate(p, axis=0)
r = np.array(r).reshape(-1,1)
print(p.shape, r.shape)

(5612, 1800, 1) (5612, 1)


In [8]:
scaler = MinMaxScaler()
scaled_p = np.array([scaler.fit_transform(pleth) for pleth in p])
print(scaled_p.shape)

(5612, 1800, 1)


In [9]:
output = model(scaled_p)
print(output.shape)

(5612, 1800, 320)
(5612, 10)
(5612, 1800, 320)
(5612, 10)


2023-09-25 18:29:04.817351: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8700
2023-09-25 18:29:20.642602: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.41GiB (rounded to 2586009600)requested by op expanding_path/StatefulPartitionedCall/conv1d_transpose_1/conv1d_transpose
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-09-25 18:29:20.642711: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-09-25 18:29:20.642746: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 362, Chunks in use: 355. 90.5KiB allocated for chunks. 88.8KiB in use in bin. 49.1KiB client-requested in use in bin.
2023-09-25 18:29:20.642768: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512

ResourceExhaustedError: Exception encountered when calling layer 'resp_net' (type RespNet).

Graph execution error:

Detected at node 'conv1d_transpose_1/conv1d_transpose' defined at (most recent call last):
    File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/root/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/root/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/root/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/root/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.10/asyncio/base_events.p

In [9]:
tf_p = tf.data.Dataset.from_tensor_slices((scaled_p[:256], r[:256])).batch(256)

In [10]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=keras.losses.MeanAbsoluteError(),
    metrics=keras.metrics.MeanAbsoluteError()
)  

In [11]:
model.fit(tf_p, epochs=10)

Epoch 1/10


2023-09-25 17:24:38.186127: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [256,1]
	 [[{{node Placeholder/_1}}]]


(None, 1800, 320)
(None, 1800, 320)


2023-09-25 17:24:42.930498: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/conv1d_33/Conv1D/BatchToSpaceND_grad/SpaceToBatchND/conv1d_33/Conv1D/BatchToSpaceND/block_shape' with dtype int32 and shape [1]
	 [[{{node gradients/conv1d_33/Conv1D/BatchToSpaceND_grad/SpaceToBatchND/conv1d_33/Conv1D/BatchToSpaceND/block_shape}}]]
2023-09-25 17:24:42.930619: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/conv1d_33/Conv1D/BatchToSpaceND_grad/SpaceToBatchND/conv1d_33/Conv1D/BatchToSpaceND/crops' with dtype int32 and shape [1,2]
	 [[{{node gradients/conv1d_33/Conv1D/BatchToSpaceND_grad/SpaceToBatchND/conv1d_3

UnimplementedError: Graph execution error:

Detected at node 'conv1d_5/Conv1D' defined at (most recent call last):
    File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/root/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/root/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/root/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/root/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
      self._run_once()
    File "/usr/lib/python3.10/asyncio/bas

In [6]:
models = [RespNet]
results = np.array([('RespNet', cross_validation(model, model_name='RespNet', dataset=stmary, dataset_name='stmary', n_splits=5, batch_size=BATCH_SIZE, lr=LR)) for i, model in enumerate(models)])

2023-09-25 16:59:55.477746: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 98 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:73:00.0, compute capability: 7.5
2023-09-25 16:59:55.498871: I tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:736] failed to allocate 98.94MiB (103743488 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-09-25 16:59:55.499676: I tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:736] failed to allocate 89.04MiB (93369344 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory


Overall: (4483, 1800, 1), (4483, 1)
Overall: (1129, 1800, 1), (1129, 1)
Epoch 1/1000


2023-09-25 16:59:57.513968: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [4483,1]
	 [[{{node Placeholder/_1}}]]


(None, 1800, 320)
(None, 1800, 320)


2023-09-25 17:00:01.541176: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/expanding_path/d_res_inc_block_4/conv1d_33/Conv1D/BatchToSpaceND_grad/SpaceToBatchND/expanding_path/d_res_inc_block_4/conv1d_33/Conv1D/BatchToSpaceND/block_shape' with dtype int32 and shape [1]
	 [[{{node gradients/expanding_path/d_res_inc_block_4/conv1d_33/Conv1D/BatchToSpaceND_grad/SpaceToBatchND/expanding_path/d_res_inc_block_4/conv1d_33/Conv1D/BatchToSpaceND/block_shape}}]]
2023-09-25 17:00:01.541274: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/expanding_path/d_res_inc_block_4/conv1d_33/Conv1D/BatchToSpaceND_grad/Spa

UnimplementedError: Graph execution error:

Detected at node 'contracting_path/d_res_inc_block/conv1d/Conv1D' defined at (most recent call last):
    File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/root/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/root/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/root/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/root/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
      self._run_once()
    File "

In [ ]:
stmary_test = np.load(f'{DATA_SAVE_PATH}/230920/stmary-test_dataset.npy', allow_pickle=True)
print(stmary_test.shape)